In [1]:
import pandas as pd

file_path = '../data/interim/data.csv'

df = pd.read_csv(file_path)

# Convert Year and Grade to integers
df['Year'] = df['Year'].astype(int)
df['Grade'] = df['Grade'].astype(int)

df.head()

,Grade,Year,Time,School,Name
0,12,2011,936.15,A1,ID1
1,11,2010,907.11,A1,ID1
2,9,2010,927.28,A7,ID10
3,10,2010,1020.66,A2,ID100
4,12,2014,1063.18,D2,ID1000


In [2]:
def increment_grades(df):
    df['Grade'] = df['Grade'] + (df['Year'] - df.groupby('Name')['Year'].shift(1).fillna(df['Year']) - 1)
    df = df[df['Grade'] <= 12]
    return df

df = increment_grades(df)

# Display the updated dataframe
df.head()

,Grade,Year,Time,School,Name
0,11.0,2011,936.15,A1,ID1
1,9.0,2010,907.11,A1,ID1
2,8.0,2010,927.28,A7,ID10
3,9.0,2010,1020.66,A2,ID100
4,11.0,2014,1063.18,D2,ID1000


In [3]:
import numpy as np

def impute_new_runners(df, year):
    schools = df['School'].unique()
    new_runners = []
    
    for school in schools:
        school_df = df[df['School'] == school]
        existing_runners = school_df[school_df['Year'] == year]
        
        if len(existing_runners) < 10:
            missing_runners = 10 - len(existing_runners)
            historical_times = school_df[school_df['Year'] < year]['Time']
            if not historical_times.empty:
                imputed_times = np.random.choice(historical_times, missing_runners, replace=True)
                for time in imputed_times:
                    new_runners.append({
                        'Name': 'Imputed_Runner',
                        'Grade': 9,
                        'School': school,
                        'Year': year,
                        'Time': time
                    })
    
    return pd.DataFrame(new_runners)

new_runners_2018 = impute_new_runners(df, 2018)
new_runners_2019 = impute_new_runners(df, 2019)

# Add new runners to the dataframe
df = pd.concat([df, new_runners_2018, new_runners_2019])

# Display the dataframe with new runners
df.head()

,Grade,Year,Time,School,Name
0,11.0,2011,936.15,A1,ID1
1,9.0,2010,907.11,A1,ID1
2,8.0,2010,927.28,A7,ID10
3,9.0,2010,1020.66,A2,ID100
4,11.0,2014,1063.18,D2,ID1000


In [4]:
# Save the updated dataframe to a new file
file_path = '../data/processed/data.csv'

df.to_csv(file_path, index=False)

# Display the file path
file_path

'../data/processed/data.csv'